# Proof of concept Machine Learning model for coin classification

I tried to do a very simple proof of concept using the pulled out dataset of coin images. The goal was to classify the coins based on their visual features. This is going to use labelled images that have been broken into obverse and reverse sides and attached to a type where available. 

First make sure you have OpenCV installed and ready to use. The code samples below are juts going to show to split the images from the small dataset into their two sides, ready to use. The full model I used has been trained on all the images, and that will be used later. 

In [2]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
Note: you may need to restart the kernel to use updated packages.


Let's now split the images. 

In [3]:
import os
import cv2

# --- Configuration ---
# The folder where your combined coin images are located
input_folder = './data/downloaded_images'
# The folder where the split images will be saved
output_folder = './data/split_images'

# --- Main Script ---
# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: '{output_folder}'")

# Get a list of all files in the input folder
all_files = os.listdir(input_folder)

print(f"Found {len(all_files)} files in '{input_folder}'. Starting to process...")

for filename in all_files:
    # Check if the file is a common image format
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(input_folder, filename)
        
        # Read the image
        img = cv2.imread(file_path)

        # Check if the image was loaded correctly
        if img is None:
            print(f"Warning: Could not read image '{filename}'. Skipping.")
            continue
        
        # Get the dimensions of the image
        height, width, _ = img.shape
        
        # Check if the image is wide enough to be split
        if width <= 10:  # A small threshold to prevent errors on tiny files
            print(f"Warning: Image '{filename}' is too narrow to split. Skipping.")
            continue

        # Split the image into two halves
        split_point = width // 2
        obverse_img = img[:, :split_point]
        reverse_img = img[:, split_point:]
        
        # Create new filenames for the split images
        base_name, ext = os.path.splitext(filename)
        obverse_filename = f"{base_name}_obverse{ext}"
        reverse_filename = f"{base_name}_reverse{ext}"
        
        # Define the full paths to save the new images
        obverse_path = os.path.join(output_folder, obverse_filename)
        reverse_path = os.path.join(output_folder, reverse_filename)

        # Save the new images
        cv2.imwrite(obverse_path, obverse_img)
        cv2.imwrite(reverse_path, reverse_img)
        
        print(f"Successfully split and saved '{filename}' into '{obverse_filename}' and '{reverse_filename}'.")

print("\nImage splitting process complete.")


Created output folder: './data/split_images'
Found 78 files in './data/downloaded_images'. Starting to process...
Successfully split and saved 'HAMP-B3BE56_67ab8670dfebc.jpg' into 'HAMP-B3BE56_67ab8670dfebc_obverse.jpg' and 'HAMP-B3BE56_67ab8670dfebc_reverse.jpg'.
Successfully split and saved 'BERK-DF77F7_67d402122a7df.jpg' into 'BERK-DF77F7_67d402122a7df_obverse.jpg' and 'BERK-DF77F7_67d402122a7df_reverse.jpg'.
Successfully split and saved 'SUR-E9366E_67ce937b6b569.jpg' into 'SUR-E9366E_67ce937b6b569_obverse.jpg' and 'SUR-E9366E_67ce937b6b569_reverse.jpg'.
Successfully split and saved 'SF-27EB9B_685034acf0128.jpg' into 'SF-27EB9B_685034acf0128_obverse.jpg' and 'SF-27EB9B_685034acf0128_reverse.jpg'.
Successfully split and saved 'WMID-09A2CE_6819dbda61c89.jpg' into 'WMID-09A2CE_6819dbda61c89_obverse.jpg' and 'WMID-09A2CE_6819dbda61c89_reverse.jpg'.
Successfully split and saved 'SUR-ADE232_67dade99159e6.jpg' into 'SUR-ADE232_67dade99159e6_obverse.jpg' and 'SUR-ADE232_67dade99159e6_revers

Now the images are ready, you can train the model. This will be a very simple proof of concept. 

In [8]:
pip install numpy tensorflow scikit-learn   

Defaulting to user installation because normal site-packages is not writeable
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 29.8 MB/s  0:00:00eta 0:00:01
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 44.1 MB/s  0:00:00 eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

def load_images_and_labels(image_dir, df, image_size=(128, 128), target_column='rrcID'):
    """
    Loads and preprocesses images, matching them to labels from a DataFrame.
    
    Args:
        image_dir (str): Path to the directory containing images.
        df (pd.DataFrame): DataFrame with coin data and filenames.
        image_size (tuple): Desired size for resizing images.
        target_column (str): The column in the DataFrame to use for labels.
    
    Returns:
        tuple: A tuple containing (images_array, labels_array).
    """
    images = []
    labels = []
    
    # Create a mapping from a sanitized filename to the target label
    # This assumes 'filename' in the CSV matches the base part of the image filename.
    df['filename_base'] = df['filename'].str.lower().str.replace('.jpg', '').str.replace('.jpeg', '')
    label_map = df.set_index('filename_base')[target_column].to_dict()
    
    # List all files in the image directory
    all_img_files = os.listdir(image_dir)
    print(f"Found {len(all_img_files)} files in the directory.")
    
    for img_filename in all_img_files:
        # Sanitize the image filename to match the format in the DataFrame
        # Assumes the filename format is something like 'rrcID_obverse.jpg'
        base_filename_parts = img_filename.lower().split('_')
        if len(base_filename_parts) > 1 and (base_filename_parts[-1].endswith('.jpg') or base_filename_parts[-1].endswith('.jpeg')):
            # The base filename is everything before the last underscore and file extension
            base_filename = '_'.join(base_filename_parts[:-1])
        else:
            # Handle cases where there is no underscore, like 'rrcID.jpg'
            base_filename = os.path.splitext(img_filename)[0].lower()
            
        # Check if this base filename exists in the mapping
        if base_filename in label_map:
            label = label_map[base_filename]
            
            if pd.isna(label):
                print(f"Skipping image '{img_filename}' due to NaN label.")
                continue

            img_path = os.path.join(image_dir, img_filename)
            
            try:
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Warning: Could not read image at {img_path}")
                    continue
                img = cv2.resize(img, image_size)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Could not load image {img_filename}: {e}")
        else:
            print(f"No matching label found for image: {img_filename}")
            
    images = np.array(images, dtype='float32') / 255.0
    labels = np.array(labels)
    
    return images, labels

def build_cnn_model(input_shape, num_classes):
    """
    Builds a CNN model for multi-class classification.
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
                  
    return model

if __name__ == "__main__":
    # --- Configuration ---
    image_directory = "./data/split_images"
    csv_file_path = "./data/reece1.csv"
    model_output_path = "./models/coin_classifier_rrcID.keras"

    # --- Data Preprocessing and Loading ---
    try:
        df_labels = pd.read_csv(csv_file_path)
        df_labels = df_labels.dropna(subset=['rrcID'])
    except FileNotFoundError:
        print(f"Error: The CSV file '{csv_file_path}' was not found.")
        exit()

    le = LabelEncoder()
    # Fit the encoder on the entire 'rrcID' column before dropping any data
    le.fit(df_labels['rrcID'])
    
    # Now, load images and map to the encoded labels
    X, y_labels = load_images_and_labels(image_directory, df_labels, target_column='rrcID')
    print(f"\nLoaded {len(X)} images.")
    if len(X) == 0:
        print("No images found or loaded. Please check your data directory and filename format.")
        exit()
    
    # Transform the loaded labels to their numerical representation
    y_encoded = le.transform(y_labels)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    # --- Model Building and Training ---
    input_shape = X_train[0].shape
    # Use the number of classes from the fitted LabelEncoder
    num_classes = len(le.classes_) 
    
    model = build_cnn_model(input_shape, num_classes)
    
    print("\nStarting model training...")
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    
    # --- Evaluation and Saving ---
    print("\nEvaluating model on test data...")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nTest Accuracy: {accuracy*100:.2f}%")

    model.save(model_output_path)
    print(f"\nModel saved to '{model_output_path}'")
    
    label_mapping = dict(zip(le.transform(le.classes_), le.classes_))
    print("\nLabel Mapping:", label_mapping)

Found 156 files in the directory.
No matching label found for image: WMID-09A2CE_6819dbda61c89_obverse.jpg
No matching label found for image: SF-C00464_686fa5b01dd78_reverse.jpg
No matching label found for image: SUR-ADE232_67dade99159e6_reverse.jpg
No matching label found for image: WREX-88411A_67c8865c65ca8_obverse.jpg
No matching label found for image: SF-3016DF_6853c3a9da387_obverse.jpg
No matching label found for image: YORYM-5DC6ED_68501ee5d96eb_reverse.jpg
No matching label found for image: SF-B2A999_6825f6f19caa2_reverse.jpg
No matching label found for image: SF-27EB9B_685034acf0128_reverse.jpg
No matching label found for image: BUC-134FE1_68513593a9599_obverse.jpg
No matching label found for image: SF-3505CE_6853c190126a2_reverse.jpg
No matching label found for image: GAT-06A665_68b06c8915771_obverse.jpg
No matching label found for image: BUC-D0BC14_67fd0c3cf3b68_obverse.jpg
No matching label found for image: OXON-B1F617_681b278a218b6_reverse.jpg
No matching label found for im

/Users/dejp3/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.0735 - loss: 3.5337 - val_accuracy: 0.0000e+00 - val_loss: 4.5626
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0559 - loss: 3.7118 - val_accuracy: 0.0000e+00 - val_loss: 3.2995
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0559 - loss: 3.1374 - val_accuracy: 0.0000e+00 - val_loss: 3.3148
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0839 - loss: 3.1010 - val_accuracy: 0.0000e+00 - val_loss: 3.3747
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1294 - loss: 3.0025 - val_accuracy: 0.0000e+00 - val_loss: 3.5123
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3076 - loss: 2.8408 - val_accuracy: 0.0000e+00 - val_loss: 3.9457
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1957 - loss: 2.6917 - val_accuracy: 0.0000e+00 - val_loss: 3.9305
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3673 - loss: 2.5037 - val_accuracy: 0.0000e+

So, let's proceed with using the trained model to make predictions on new images. We're going to create a function that takes a directory of new images, preprocesses them, and then uses the model to predict their classes. I've added a few coins from CRRO to use for testing.

In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def predict_batch_of_images(model_path, image_dir, le, image_size=(128, 128)):
    """
    Loads a trained model and predicts the top 3 classes for all images in a folder.
    """
    try:
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    predictions = []
    
    # Define valid image extensions
    valid_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')

    # Get a list of all files in the directory
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(image_dir, filename)
            
            # Load and preprocess the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Warning: Could not read image {filename}. Skipping.")
                continue
            
            img = cv2.resize(img, image_size)
            img = np.array(img, dtype='float32') / 255.0
            img = np.expand_dims(img, axis=0)  # Add a batch dimension

            # Make a prediction
            preds_array = model.predict(img)
            
            # Get the indices of the top 3 predictions
            top3_indices = np.argsort(preds_array[0])[-3:][::-1]
            
            # Inverse transform the indices to get the original rrcIDs
            top3_labels = le.inverse_transform(top3_indices)
            
            # Get the corresponding confidence scores
            top3_confidences = preds_array[0][top3_indices]
            
            result_entry = {
                "filename": filename,
                "predictions": []
            }
            
            for label, confidence in zip(top3_labels, top3_confidences):
                result_entry['predictions'].append({
                    "rrcID": label,
                    "confidence": f"{confidence*100:.2f}%"
                })
            
            predictions.append(result_entry)
    
    return predictions

if __name__ == "__main__":
    # --- Configuration ---
    # Path to the saved model file
    model_path = "./models/coin_classifier_rrcID.keras"
    
    # Path to the folder containing the images you want to predict
    batch_image_folder = "./data/coins_to_classify"
    
    # Path to your original CSV file
    csv_file_path = "./data/reece1.csv"
    
    # --- Label Encoder Setup ---
    # The LabelEncoder must be fitted on the same data as during training.
    try:
        df_labels = pd.read_csv(csv_file_path)
        df_labels = df_labels.dropna(subset=['rrcID'])
        le = LabelEncoder()
        le.fit(df_labels['rrcID'])
    except FileNotFoundError:
        print(f"Error: The CSV file '{csv_file_path}' was not found. Cannot load the label encoder.")
        exit()

    # --- Run Batch Prediction ---
    print(f"Starting prediction for images in '{batch_image_folder}'...")
    results = predict_batch_of_images(model_path, batch_image_folder, le)

    if results:
        print("\n--- Prediction Results ---")
        for result in results:
            print(f"File: {result['filename']}")
            for pred in result['predictions']:
                print(f"  - Predicted RRC ID: {pred['rrcID']} (Confidence: {pred['confidence']})")
            print() # Print a blank line for readability
    else:
        print("No eligible images found or predictions could not be made.")


Starting prediction for images in './data/coins_to_classify'...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

--- Prediction Results ---
File: large_00624497_001.jpg
  - Predicted RRC ID: rrc-374.1 (Confidence: 15.87%)
  - Predicted RRC ID: rrc-392.1b (Confidence: 14.96%)
  - Predicted RRC ID: rrc-391.3 (Confidence: 13.63%)

File: 1937.158.67.obv.width175.jpg
  - Predicted RRC ID: rrc-374.1 (Conf

These are going to be low confidence for that trained model due to the differences in the coin images from CRRO compared to the training dataset. So let's rerun that against the larger model that was trained and uploaded off the entire Reece Period 1 dataset.

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def predict_batch_of_images(model_path, image_dir, le, image_size=(128, 128)):
    """
    Loads a trained model and predicts the top 3 classes for all images in a folder.
    """
    try:
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    predictions = []
    
    # Define valid image extensions
    valid_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')

    # Get a list of all files in the directory
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(image_dir, filename)
            
            # Load and preprocess the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Warning: Could not read image {filename}. Skipping.")
                continue
            
            img = cv2.resize(img, image_size)
            img = np.array(img, dtype='float32') / 255.0
            img = np.expand_dims(img, axis=0)  # Add a batch dimension

            # Make a prediction
            preds_array = model.predict(img)
            
            # Get the indices of the top 3 predictions
            top3_indices = np.argsort(preds_array[0])[-3:][::-1]
            
            # Inverse transform the indices to get the original rrcIDs
            top3_labels = le.inverse_transform(top3_indices)
            
            # Get the corresponding confidence scores
            top3_confidences = preds_array[0][top3_indices]
            
            result_entry = {
                "filename": filename,
                "predictions": []
            }
            
            for label, confidence in zip(top3_labels, top3_confidences):
                result_entry['predictions'].append({
                    "rrcID": label,
                    "confidence": f"{confidence*100:.2f}%"
                })
            
            predictions.append(result_entry)
    
    return predictions

if __name__ == "__main__":
    # --- Configuration ---
    # Path to the saved model file for the entire dataset
    model_path = "../models/coin_classifier_rrcID.keras"
    
    # Path to the folder containing the images you want to predict
    batch_image_folder = "./data/coins_to_classify"
    
    # Path to your entire CSV file from Reece 1
    csv_file_path = "../data/reece1.csv"
    
    # --- Label Encoder Setup ---
    # The LabelEncoder must be fitted on the same data as during training.
    try:
        df_labels = pd.read_csv(csv_file_path)
        df_labels = df_labels.dropna(subset=['rrcID'])
        le = LabelEncoder()
        le.fit(df_labels['rrcID'])
    except FileNotFoundError:
        print(f"Error: The CSV file '{csv_file_path}' was not found. Cannot load the label encoder.")
        exit()

    # --- Run Batch Prediction ---
    print(f"Starting prediction for images in '{batch_image_folder}'...")
    results = predict_batch_of_images(model_path, batch_image_folder, le)

    if results:
        print("\n--- Prediction Results ---")
        for result in results:
            print(f"File: {result['filename']}")
            for pred in result['predictions']:
                print(f"  - Predicted RRC ID: {pred['rrcID']} (Confidence: {pred['confidence']})")
            print() # Print a blank line for readability
    else:
        print("No eligible images found or predictions could not be made.")


Starting prediction for images in './data/coins_to_classify'...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

--- Prediction Results ---
File: large_00624497_001.jpg
  - Predicted RRC ID: rrc-412.1 (Confidence: 58.97%)
  - Predicted RRC ID: rrc-544.14 (Confidence: 21.35%)
  - Predicted RRC ID: rrc-544.13 (Confidence: 8.40%)

File: 1937.158.67.obv.width175.jpg
  - Predicted RRC ID: rrc-443.1 (Conf

Slightly better results, but as PAS images are pretty variable and numbers so small, this won't be an amazing test. I would probably take the CRRO coins and reverse what I'm doing!